In [10]:
import requests
import json

def test_local_server():
    """Test local server trước khi deploy với form mới"""
    base_url = "http://localhost:8080"  # Local server
    
    # Test health
    print("=== TESTING LOCAL SERVER ===")
    try:
        health_response = requests.get(f"{base_url}/health", timeout=5)
        print(f"Health: {health_response.status_code} - {health_response.json()}")
    except Exception as e:
        print(f"❌ Local server not running: {e}")
        print("Chạy: npm run dev")
        return
    
    # Test process document với form mới
    test_data = {
        "recordId": "recuPj9iviBezQ",
        "appToken": "Z2cxb4FTWaYhXTsJdCGlDAYYgYq",
        "tableID": "tblpzAwC8Ol4VWmZ",
        "idFieldName": "ID",
        "vanBanCap1FieldName": "Văn bản cấp 1",
        "vanBanCap2FieldName": "Văn bản cấp 2",  # Optional
        "vanBanCap3FieldName": "Văn bản cấp 3",  # Optional
        "ngayThangNamVanBanFieldName": "Ngày tháng năm văn bản",  # Optional
        "phapNhanAtinoFieldName": "Pháp nhân Atino",  # Optional
        "congTyDoiTacFieldName": "Công ty đối tác",  # Optional
        "mucDoUuTienFieldName": "Mức độ ưu tiên",  # Optional
        "ghiChuFieldName": "Ghi chú",  # Optional
        "giaTriHopDongFieldName": "Giá trị hợp đồng/đơn hàng",  # Optional
        "giaTriThueMatBangFieldName": "Giá trị thuê mặt bằng/tháng",  # Optional
        "taiLieuDinhKemFieldName": "Tài liệu đính kèm",
        "creatorOpenId": "ou_41ff6114c70aef421d5e9d70953bd208"
    }
    
    print("\n=== TEST DATA ===")
    print(f"Record ID: {test_data['recordId']}")
    print(f"App Token: {test_data['appToken']}")
    print(f"Table ID: {test_data['tableID']}")
    print(f"Required fields: ID, Văn bản cấp 1, Tài liệu đính kèm")
    print(f"Optional fields: {len([k for k in test_data.keys() if 'FieldName' in k]) - 3} trường")
    
    try:
        print("\n=== SENDING REQUEST ===")
        response = requests.post(f"{base_url}/process-document", 
            json=test_data, 
            timeout=60,  # Tăng timeout vì có nhiều trường hơn
            headers={'Content-Type': 'application/json'}
        )
        
        print(f"\nProcess Document Status: {response.status_code}")
        
        if response.status_code == 200:
            result = response.json()
            print("✅ SUCCESS!")
            print(f"Instance Code: {result.get('instanceCode', 'N/A')}")
            print(f"Message: {result.get('message', 'N/A')}")
            print(f"Document Count: {result.get('documentCount', 0)}")
            
            # In thông tin chi tiết về document
            if result.get('documentInfo'):
                print("\n📄 Document Info:")
                doc_info = result['documentInfo']
                for key, value in doc_info.items():
                    if value:  # Chỉ in những field có giá trị
                        print(f"  - {key}: {value}")
            
            # In uploaded codes
            if result.get('uploadedCodes'):
                print(f"\n📎 Uploaded Files: {len(result['uploadedCodes'])} files")
                for i, code in enumerate(result['uploadedCodes'], 1):
                    print(f"  {i}. {code}")
                    
        else:
            print("❌ FAILED!")
            try:
                error_data = response.json()
                print(f"Error: {error_data.get('message', 'Unknown error')}")
                if error_data.get('errorDetails'):
                    print(f"Details: {error_data['errorDetails']}")
            except:
                print(f"Raw response: {response.text}")
        
    except requests.exceptions.Timeout:
        print("❌ Request timeout - Server mất quá nhiều thời gian xử lý")
    except requests.exceptions.ConnectionError:
        print("❌ Connection error - Không thể kết nối đến server")
    except Exception as e:
        print(f"❌ Local test failed: {e}")

def test_with_minimal_data():
    """Test với dữ liệu tối thiểu (chỉ required fields)"""
    base_url = "http://localhost:8080"
    
    print("\n=== TESTING WITH MINIMAL DATA ===")
    
    minimal_data = {
        "recordId": "recuPj9iviBezQ",
        "appToken": "Z2cxb4FTWaYhXTsJdCGlDAYYgYq",
        "tableID": "tblpzAwC8Ol4VWmZ",
        "idFieldName": "ID",
        "vanBanCap1FieldName": "Văn bản cấp 1",  # Required
        "taiLieuDinhKemFieldName": "Tài liệu đính kèm",  # Required
        "creatorOpenId": "ou_41ff6114c70aef421d5e9d70953bd208"
    }
    
    try:
        response = requests.post(f"{base_url}/process-document", 
            json=minimal_data, 
            timeout=30
        )
        
        print(f"Minimal Test Status: {response.status_code}")
        if response.status_code == 200:
            print("✅ Minimal test passed!")
        else:
            print("❌ Minimal test failed!")
            print(f"Response: {response.text}")
            
    except Exception as e:
        print(f"❌ Minimal test error: {e}")

def test_debug_approval():
    """Test debug approval endpoint"""
    base_url = "http://localhost:8080"
    approval_code = "8838A66F-0F31-4D1E-80A9-9F067F0DCD21"
    
    print(f"\n=== TESTING DEBUG APPROVAL: {approval_code} ===")
    
    try:
        response = requests.get(f"{base_url}/debug-approval/{approval_code}", timeout=10)
        print(f"Debug Approval Status: {response.status_code}")
        
        if response.status_code == 200:
            data = response.json()
            print("✅ Debug approval success!")
            print(f"Form widgets count: {len(data.get('data', {}).get('form', {}).get('form_content', []))}")
        else:
            print("❌ Debug approval failed!")
            print(f"Response: {response.text}")
            
    except Exception as e:
        print(f"❌ Debug approval error: {e}")

if __name__ == "__main__":
    # Test full functionality
    test_local_server()
    
    # Test minimal data
    # test_with_minimal_data()
    
    # Test debug endpoint
    # test_debug_approval()
    
    print("\n=== TEST COMPLETED ===")


=== TESTING LOCAL SERVER ===
Health: 200 - {'status': 'OK', 'timestamp': '2025-06-30T07:10:22.803Z'}

=== TEST DATA ===
Record ID: recuPj9iviBezQ
App Token: Z2cxb4FTWaYhXTsJdCGlDAYYgYq
Table ID: tblpzAwC8Ol4VWmZ
Required fields: ID, Văn bản cấp 1, Tài liệu đính kèm
Optional fields: 9 trường

=== SENDING REQUEST ===

Process Document Status: 200
✅ SUCCESS!
Instance Code: E2DA752D-7908-4CAF-B513-C26C959C150E
Message: Đã tạo thành công đơn phê duyệt văn bản với 1 tài liệu đính kèm
Document Count: 1

📄 Document Info:
  - vanBanCap1: Văn bản đối ngoại – giao dịch với bên ngoài
  - vanBanCap2: Hợp đồng kinh tế
  - vanBanCap3: Hợp đồng thuê mặt bằng
  - phapNhanAtino: Gia Bảo
  - congTyDoiTac: Nguyễn Phương Thúy
  - mucDoUuTien: Không gấp
  - giaTriHopDong: 300000000
  - giaTriThueMatBang: 30000000
  - ngayThangNamVanBan: 1750179600000

📎 Uploaded Files: 1 files
  1. 9F6C8664-9898-411E-8295-4F808227DF73

=== TEST COMPLETED ===


In [9]:
import requests
import json

def test_cloud_server():
    """Test cloud server trên Google Cloud Run"""
    base_url = "https://document-approval-processor-858039461446.asia-southeast1.run.app"
    
    # Test health
    print("=== TESTING CLOUD SERVER ===")
    try:
        health_response = requests.get(f"{base_url}/health", timeout=10)
        print(f"Health: {health_response.status_code} - {health_response.json()}")
    except Exception as e:
        print(f"❌ Cloud server error: {e}")
        return
    
    # Test debug approval
    print("\n=== TESTING DEBUG APPROVAL ===")
    try:
        debug_response = requests.get(
            f"{base_url}/debug-approval/80979F0E-15E1-479E-8865-DC291883E5ED", 
            timeout=15
        )
        print(f"Debug Approval Status: {debug_response.status_code}")
        if debug_response.status_code == 200:
            print("✓ Debug approval endpoint working")
        else:
            print(f"Debug response: {debug_response.text}")
    except Exception as e:
        print(f"❌ Debug approval failed: {e}")
    
    # Test process document
    print("\n=== TESTING PROCESS DOCUMENT ===")
    test_data = {
        "recordId": "recuPj9iviBezQ",
        "appToken": "Z2cxb4FTWaYhXTsJdCGlDAYYgYq",
        "tableID": "tblpzAwC8Ol4VWmZ",
        "idFieldName": "ID",
        "loaiVanBanFieldName": "Loại văn bản - con",
        "hoSoDinhKemFieldName": "Tài liệu đính kèm",
        "creatorOpenId": "ou_41ff6114c70aef421d5e9d70953bd208"
    }
    
    try:
        print("Sending request to cloud server...")
        response = requests.post(
            f"{base_url}/process-document", 
            json=test_data, 
            timeout=60,  # Tăng timeout cho cloud
            headers={'Content-Type': 'application/json'}
        )
        
        print(f"\nProcess Document Status: {response.status_code}")
        
        if response.status_code == 200:
            print("✓ SUCCESS!")
            print(f"Response: {json.dumps(response.json(), indent=2, ensure_ascii=False)}")
        else:
            print(f"❌ FAILED!")
            print(f"Error Response: {response.text}")
            
    except requests.exceptions.Timeout:
        print("❌ Request timeout - Cloud function might be cold starting")
    except Exception as e:
        print(f"❌ Cloud test failed: {e}")


if __name__ == "__main__":
    test_cloud_server() 



=== TESTING CLOUD SERVER ===
Health: 200 - {'status': 'OK', 'timestamp': '2025-06-30T07:10:15.150Z'}

=== TESTING DEBUG APPROVAL ===
Debug Approval Status: 200
✓ Debug approval endpoint working

=== TESTING PROCESS DOCUMENT ===
Sending request to cloud server...

Process Document Status: 500
❌ FAILED!
Error Response: {"success":false,"message":"Lỗi: Trường \"Loại văn bản - con\" không có giá trị hoặc rỗng","uploadedCodes":[],"documentCount":0,"errorDetails":"Trường \"Loại văn bản - con\" không có giá trị hoặc rỗng"}


In [ ]:
# import json

# # JSON response từ API
# response_data = {
#     "body": {
#         "code": 0,
#         "data": {
#             "user_list": [
#                 {
#                     "status": {
#                         "is_exited": False,
#                         "is_frozen": False,
#                         "is_resigned": False,
#                         "is_unjoin": False,
#                         "is_activated": True
#                     },
#                     "user_id": "ou_7724fc05f615b0dfb76403574ad717e7",
#                     "email": "le.syduc.993@gmail.com"
#                 }
#             ]
#         },
#         "msg": "success"
#     },
#     "headers": {
#         # ... headers data
#     },
#     "status_code": 200
# }

# # Cách 1: Truy cập trực tiếp
# try:
#     user_id = response_data['body']['data']['user_list'][0]['user_id']
#     print(f"User ID: {user_id}")
# except (KeyError, IndexError) as e:
#     print(f"Lỗi khi lấy user_id: {e}")

# # Cách 2: Kiểm tra an toàn hơn
# def get_user_id(response):
#     try:
#         body = response.get('body', {})
#         data = body.get('data', {})
#         user_list = data.get('user_list', [])
        
#         if user_list and len(user_list) > 0:
#             return user_list[0].get('user_id')
#         else:
#             return None
#     except Exception as e:
#         print(f"Lỗi: {e}")
#         return None

# # Sử dụng
# user_id = get_user_id(response_data)
# if user_id:
#     print(f"User ID tìm thấy: {user_id}")
# else:
#     print("Không tìm thấy user_id")

# # Cách 3: Lấy tất cả thông tin user
# def get_all_users_info(response):
#     try:
#         user_list = response['body']['data']['user_list']
#         users_info = []
        
#         for user in user_list:
#             user_info = {
#                 'user_id': user.get('user_id'),
#                 'email': user.get('email'),
#                 'is_activated': user.get('status', {}).get('is_activated', False)
#             }
#             users_info.append(user_info)
        
#         return users_info
#     except Exception as e:
#         print(f"Lỗi khi lấy thông tin users: {e}")
#         return []

# # Sử dụng
# all_users = get_all_users_info(response_data)
# for user in all_users:
#     print(f"User ID: {user['user_id']}, Email: {user['email']}, Activated: {user['is_activated']}")


User ID: ou_7724fc05f615b0dfb76403574ad717e7
User ID tìm thấy: ou_7724fc05f615b0dfb76403574ad717e7
User ID: ou_7724fc05f615b0dfb76403574ad717e7, Email: le.syduc.993@gmail.com, Activated: True


In [ ]:
import requests
import json
import traceback
from datetime import datetime
from typing import Dict, Any, Optional, List

class LarkbaseConfig:
    def __init__(self, app_id=None, app_secret=None, api_endpoint=None):
        self.app_id = app_id or 'cli_a7fab27260385010'
        self.app_secret = app_secret or 'Zg4MVcFfiOu0g09voTcpfd4WGDpA0Ly5'
        self.api_endpoint = api_endpoint or 'https://open.larksuite.com/open-apis'

    def to_dict(self) -> Dict:
        return {
            'app_id': self.app_id,
            'app_secret': self.app_secret,
            'api_endpoint': self.api_endpoint
        }

class LarkbaseAuthenticator:
    def __init__(self, config: LarkbaseConfig):
        self.config = config

    def authenticate(self) -> Optional[str]:
        try:
            url = f"{self.config.api_endpoint}/auth/v3/tenant_access_token/internal"
            response = requests.post(url, json={
                'app_id': self.config.app_id,
                'app_secret': self.config.app_secret
            })
            response.raise_for_status()
            data = response.json()
            if data.get('code') == 0:
                return data.get('tenant_access_token')
            else:
                print(f"Lỗi API: {data.get('msg', 'Không xác định')}")
                return None
        except Exception as e:
            print(f"Lỗi xác thực: {str(e)}")
            return None

class LarkBaseManager:
    def __init__(self, config: LarkbaseConfig, access_token: str):
        self.config = config
        self.access_token = access_token

    def get_record(self, app_token: str, table_id: str, record_id: str) -> Dict[str, Any]:
        url = f"{self.config.api_endpoint}/bitable/v1/apps/{app_token}/tables/{table_id}/records/{record_id}"
        headers = {
            "Authorization": f"Bearer {self.access_token}",
            "Content-Type": "application/json"
        }
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            return response.json()
        else:
            raise Exception(f"Lỗi khi lấy record: {response.status_code} - {response.text}")

class LarkDriveManager:
    def __init__(self, config: LarkbaseConfig, access_token: str):
        self.config = config
        self.access_token = access_token

    def download_file(self, file_token: str) -> bytes:
        url = f"{self.config.api_endpoint}/drive/v1/medias/{file_token}/download"
        headers = {
            "Authorization": f"Bearer {self.access_token}"
        }
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            return response.content
        else:
            raise Exception(f"Lỗi khi tải file: {response.status_code} - {response.text}")

class LarkMessengerManager:
    def __init__(self, config: LarkbaseConfig, access_token: str):
        self.config = config
        self.access_token = access_token

    def upload_image(self, file_content: bytes, filename: str = "image.png") -> str:
        url = f"{self.config.api_endpoint}/im/v1/images"
        headers = {
            "Authorization": f"Bearer {self.access_token}"
        }
        files = {
            'image': (filename, file_content, 'image/png')
        }
        data = {
            'image_type': 'message'
        }
        response = requests.post(url, headers=headers, files=files, data=data)
        if response.status_code == 200:
            result = response.json()
            image_key = result.get('data', {}).get('image_key', '')
            return image_key
        else:
            raise Exception(f"Lỗi khi upload image: {response.status_code} - {response.text}")

# class MessageCardBuilder:
#     """Xây dựng message card với bảng doanh thu cửa hàng, tỉ lệ hoàn thành target và hỗ trợ đính kèm ảnh"""

#     @staticmethod
#     def build_table_column_set(data_rows: List[Dict[str, str]]) -> List[Dict]:
#         """Tạo bảng dạng column set với căn chỉnh đều, có thêm cột tỉ lệ hoàn thành target"""
#         table = [
#             {
#                 "tag": "column_set",
#                 "columns": [
#                     {
#                         "tag": "column",
#                         "width": "weighted",
#                         "weight": 3,
#                         "elements": [
#                             {
#                                 "tag": "div",
#                                 "text": {
#                                     "content": "🏪 **Tên cửa hàng**",
#                                     "tag": "lark_md"
#                                 }
#                             }
#                         ]
#                     },
#                     {
#                         "tag": "column",
#                         "width": "weighted",
#                         "weight": 2,
#                         "elements": [
#                             {
#                                 "tag": "div",
#                                 "text": {
#                                     "content": "💰 **Doanh thu thực tế**",
#                                     "tag": "lark_md"
#                                 }
#                             }
#                         ]
#                     },
#                     {
#                         "tag": "column",
#                         "width": "weighted",
#                         "weight": 2,
#                         "elements": [
#                             {
#                                 "tag": "div",
#                                 "text": {
#                                     "content": "📊 **Doanh thu TB 30 ngày**",
#                                     "tag": "lark_md"
#                                 }
#                             }
#                         ]
#                     },
#                     {
#                         "tag": "column",
#                         "width": "weighted",
#                         "weight": 2,
#                         "elements": [
#                             {
#                                 "tag": "div",
#                                 "text": {
#                                     "content": "🎯 **Tỉ lệ hoàn thành target**",
#                                     "tag": "lark_md"
#                                 }
#                             }
#                         ]
#                     }
#                 ]
#             }
#         ]

#         # Thêm các hàng dữ liệu với weight tương ứng
#         for row in data_rows:
#             table.append({
#                 "tag": "column_set",
#                 "columns": [
#                     {
#                         "tag": "column",
#                         "width": "weighted",
#                         "weight": 3,
#                         "elements": [
#                             {
#                                 "tag": "div",
#                                 "text": {
#                                     "content": f"{row['store_name']}",
#                                     "tag": "lark_md"
#                                 }
#                             }
#                         ]
#                     },
#                     {
#                         "tag": "column",
#                         "width": "weighted",
#                         "weight": 2,
#                         "elements": [
#                             {
#                                 "tag": "div",
#                                 "text": {
#                                     "content": f"{row['doanh_thu_thuc_te']}",
#                                     "tag": "lark_md"
#                                 }
#                             }
#                         ]
#                     },
#                     {
#                         "tag": "column",
#                         "width": "weighted",
#                         "weight": 2,
#                         "elements": [
#                             {
#                                 "tag": "div",
#                                 "text": {
#                                     "content": f"{row['doanh_thu_tb_30_ngay']}",
#                                     "tag": "lark_md"
#                                 }
#                             }
#                         ]
#                     },
#                     {
#                         "tag": "column",
#                         "width": "weighted",
#                         "weight": 2,
#                         "elements": [
#                             {
#                                 "tag": "div",
#                                 "text": {
#                                     "content": f"{row.get('ti_le_hoan_thanh_target', '')}%",
#                                     "tag": "lark_md"
#                                 }
#                             }
#                         ]
#                     }
#                 ]
#             })
#         return table

#     @staticmethod
#     def create_summary_card(
#         title: str, 
#         data_rows: List[Dict[str, str]], 
#         image_key: Optional[str] = None
#     ) -> Dict[str, Any]:
#         """
#         Tạo message card với bảng doanh thu cửa hàng, tỉ lệ hoàn thành target và ảnh đính kèm
        
#         Args:
#             title: Tiêu đề của card
#             data_rows: Dữ liệu bảng với keys: 'store_name', 'doanh_thu_thuc_te', 'doanh_thu_tb_30_ngay', 'ti_le_hoan_thanh_target_luy_ke'
#             image_key: Key của ảnh đã upload (từ LarkMessengerManager.upload_image)
#         """
#         elements = []
#         if image_key:
#             elements.append({
#                 "tag": "img",
#                 "img_key": image_key,
#                 "alt": {
#                     "tag": "plain_text",
#                     "content": "Biểu đồ doanh thu cửa hàng"
#                 }
#             })
#         table_elements = MessageCardBuilder.build_table_column_set(data_rows)
#         elements.extend(table_elements)
#         card_template = {
#             "config": {
#                 "wide_screen_mode": True
#             },
#             "elements": elements,
#             "header": {
#                 "template": "blue",
#                 "title": {
#                     "content": title,
#                     "tag": "plain_text"
#                 }
#             }
#         }
#         return card_template


class MessageCardBuilder:
    """Xây dựng message card với bảng doanh thu cửa hàng, tỉ lệ hoàn thành target và hỗ trợ đính kèm ảnh"""

    @staticmethod
    def format_currency_short(amount):
        """Định dạng số tiền ngắn gọn"""
        if amount is None:
            return "N/A"
        try:
            amount = float(amount)
            if amount >= 1_000_000_000:
                return f"{amount/1_000_000_000:.1f}B"
            elif amount >= 1_000_000:
                return f"{amount/1_000_000:.1f}M"
            elif amount >= 1_000:
                return f"{amount/1_000:.1f}K"
            else:
                return f"{amount:,.0f}"
        except:
            return str(amount)

    @staticmethod
    def build_table_column_set(data_rows: List[Dict[str, str]]) -> List[Dict]:
        """Tạo bảng dạng column set với 5 cột được tối ưu"""
        table = [
            {
                "tag": "column_set",
                "columns": [
                    {
                        "tag": "column",
                        "width": "weighted",
                        "weight": 2.5,  # Giảm từ 3 xuống 2.5
                        "elements": [
                            {
                                "tag": "div",
                                "text": {
                                    "content": "🏪 **Cửa hàng**",  # Rút gọn từ "Tên cửa hàng"
                                    "tag": "lark_md"
                                }
                            }
                        ]
                    },
                    {
                        "tag": "column",
                        "width": "weighted",
                        "weight": 1.5,  # Giảm từ 2 xuống 1.5
                        "elements": [
                            {
                                "tag": "div",
                                "text": {
                                    "content": "💰 **DT hôm qua**",  # Rút gọn
                                    "tag": "lark_md"
                                }
                            }
                        ]
                    },
                    {
                        "tag": "column",
                        "width": "weighted",
                        "weight": 1.5,  # Giảm từ 2 xuống 1.5
                        "elements": [
                            {
                                "tag": "div",
                                "text": {
                                    "content": "📊 **DT TB 30d**",  # Rút gọn
                                    "tag": "lark_md"
                                }
                            }
                        ]
                    },
                    {
                        "tag": "column",
                        "width": "weighted",
                        "weight": 1.5,  # Giảm từ 2 xuống 1.5
                        "elements": [
                            {
                                "tag": "div",
                                "text": {
                                    "content": "💎 **Tổng DT tháng**",  # Cột mới
                                    "tag": "lark_md"
                                }
                            }
                        ]
                    },
                    {
                        "tag": "column",
                        "width": "weighted",
                        "weight": 1.5,  # Giảm từ 2 xuống 1.5
                        "elements": [
                            {
                                "tag": "div",
                                "text": {
                                    "content": "🎯 **% Target**",  # Rút gọn
                                    "tag": "lark_md"
                                }
                            }
                        ]
                    }
                ]
            }
        ]
        
        # Thêm các hàng dữ liệu với weight tương ứng
        for row in data_rows:
            table.append({
                "tag": "column_set",
                "columns": [
                    {
                        "tag": "column",
                        "width": "weighted",
                        "weight": 2.5,
                        "elements": [
                            {
                                "tag": "div",
                                "text": {
                                    "content": f"{row['store_name']}",
                                    "tag": "lark_md"
                                }
                            }
                        ]
                    },
                    {
                        "tag": "column",
                        "width": "weighted",
                        "weight": 1.5,
                        "elements": [
                            {
                                "tag": "div",
                                "text": {
                                    "content": f"{MessageCardBuilder.format_currency_short(row.get('doanh_thu_thuc_te', 0))}",
                                    "tag": "lark_md"
                                }
                            }
                        ]
                    },
                    {
                        "tag": "column",
                        "width": "weighted",
                        "weight": 1.5,
                        "elements": [
                            {
                                "tag": "div",
                                "text": {
                                    "content": f"{MessageCardBuilder.format_currency_short(row.get('doanh_thu_tb_30_ngay', 0))}",
                                    "tag": "lark_md"
                                }
                            }
                        ]
                    },
                    {
                        "tag": "column",
                        "width": "weighted",
                        "weight": 1.5,
                        "elements": [
                            {
                                "tag": "div",
                                "text": {
                                    "content": f"{MessageCardBuilder.format_currency_short(row.get('tong_doanh_thu_thang', 0))}",  # Cột mới
                                    "tag": "lark_md"
                                }
                            }
                        ]
                    },
                    {
                        "tag": "column",
                        "width": "weighted",
                        "weight": 1.5,
                        "elements": [
                            {
                                "tag": "div",
                                "text": {
                                    "content": f"{row.get('ti_le_hoan_thanh_target', 0)}%",
                                    "tag": "lark_md"
                                }
                            }
                        ]
                    }
                ]
            })
        return table

    @staticmethod
    def create_summary_card(
        title: str, 
        data_rows: List[Dict[str, str]], 
        image_key: Optional[str] = None
    ) -> Dict[str, Any]:
        """
        Tạo message card với bảng doanh thu cửa hàng 5 cột được tối ưu
        
        Args:
            title: Tiêu đề của card
            data_rows: Dữ liệu bảng với keys: 'store_name', 'doanh_thu_thuc_te', 'doanh_thu_tb_30_ngay', 'tong_doanh_thu_thang', 'ti_le_hoan_thanh_target'
            image_key: Key của ảnh đã upload
        """
        elements = []
        
        if image_key:
            elements.append({
                "tag": "img",
                "img_key": image_key,
                "alt": {
                    "tag": "plain_text",
                    "content": "Biểu đồ doanh thu cửa hàng"
                }
            })
        
        table_elements = MessageCardBuilder.build_table_column_set(data_rows)
        elements.extend(table_elements)
        
        card_template = {
            "config": {
                "wide_screen_mode": True
            },
            "elements": elements,
            "header": {
                "template": "blue",
                "title": {
                    "content": title,
                    "tag": "plain_text"
                }
            }
        }
        return card_template



class LarkBotManager:
    def __init__(self, bot_webhook: str):
        self.bot_webhook = bot_webhook

    def send_message_card(self, card_json: Dict[str, Any]) -> bool:
        payload = {
            "msg_type": "interactive",
            "card": card_json
        }
        response = requests.post(self.bot_webhook, json=payload)
        if response.status_code == 200:
            try:
                data = response.json()
                if data.get("code", 0) != 0:
                    print(f"[ERROR] Lark trả về lỗi: {data.get('msg', '')} (code: {data.get('code')})")
                    return False
                print("✅ Đã gửi message card thành công!")
                return True
            except Exception as e:
                print(f"[ERROR] Không parse được JSON response: {str(e)}")
                return False
        else:
            print(f"❌ Lỗi gửi message card: {response.status_code} - {response.text}")
            return False


🔧 Testing Google Apps Script - Hợp đồng khoán việc
🚀 Đang gửi request tạo hợp đồng...
URL: https://script.google.com/macros/s/AKfycbwcEnwV6PINoTV8ArCyVl07wBG6el1Gz34wTsT0IRgdtQiwELRoLsvbslnZScqysDew/exec
Dữ liệu: {
  "code": "HĐ20250628001",
  "date_start": "2025-07-01",
  "date_finish": "2025-12-31",
  "personnel_name": "Nguyễn Văn A",
  "personnel_code": "NV001",
  "profile.gender": "Nam",
  "profile.birthday": "1990-05-15",
  "profile.level_academic": "Đại học",
  "profile.private_code": "001234567890",
  "profile.private_code_date": "2020-01-15",
  "profile.private_code_place": "CA Hà Nội",
  "profile.job_bank_account": "1234567890",
  "profile.job_bank_id": "Vietcombank",
  "profile.place_home": "Số 123, Đường ABC, Quận XYZ, Hà Nội",
  "profile.place_current": "Số 456, Đường DEF, Quận GHI, Hà Nội",
  "profile.mobile": "0987654321",
  "profile.email": "nguyenvana@email.com",
  "profile.day_job": "26",
  "profile.salary_11": "8000000",
  "profile.salary_12": "2000000",
  "TEMPLATE_I